# Network energy

Initial implementation of the models defined in the [Carbon Impact of Video
Streaming report from The Carbon Trust (June
2021)](https://www.carbontrust.com/resources/carbon-impact-of-video-streaming).
The purpose of this notebook is to fully reproduce the model.

A breakdown of the results from the conventional model is provided in Table 4 
on pg 52 of the report.

## Imports

In [24]:
%%capture
# ^ Hide the output of this cell (must be on the first line)

# Install dependencies
%pip install -r requirements.txt

## Global parameters

In [25]:
durationStreaming = 1  # Duration of video streaming (D) Probably hours, not specified
dataTransmissionRate = (
    3  # Data transmission rate (R) GB/hr Using FHD value, not specified (pg43, Table 3)
)

## Conventional model

Carbon Trust report pg40-46.

### Global parameters

In [26]:
energyVideoStreaming = float  # Energy consumption of video streaming (E_V)
carbonVideoStreaming = float  # Carbon emissions of video streaming (C_V)
gridEmissionsFactor = 1  # ??? Emissions factor of the grid for region, n (EF_g,n)

### Data centers & CDNs

Carbon Trust value = 1 Wh/hour.

In [27]:
dataCenterEnergy = float  # Data center energy consumption (E_DC)
dataCenterIntensity = 1.3  # Data center intensity (I_DC) Wh/hr (pg43, Table 3)

In [28]:
# Equation (1)
dataCenterEnergy = dataCenterIntensity * durationStreaming
dataCenterEnergy

1.3

### Transmission - core & access

Carbon Trust value = 20 Wh/hour.

In [29]:
energyFixedC = float  # Fixed network energy consumption (E_FN)
intensityFixed = 0.0065  # Energy intensity of fixed network transmission (I_FN) kWh/GB (pg43, Table 3)
# proportionFixed = ? # proportion of viewing time over fixed network at data transmission rate R_i relative to the entire streaming service (P_FN,i)
energyMobileC = float  # Mobile network energy consumption (E_MN)
intensityMobile = (
    0.1  # Energy intensity of mobile network transmission (I_MN) kWh/GB (pg43, Table 3)
)
# proportionMobile = ? # Proportion of viewing time over mobile network at data transmission rate R_i relative to the entire streaming service (P_MN,i)

In [30]:
# Equation (2) - single viewing scenario
energyFixedC = intensityFixed * durationStreaming * dataTransmissionRate  # kWh
energyFixedCWh = energyFixedC * 1000  # Wh
energyFixedCWh  # Pg 52, Table 4 = 20 Wh/hour

19.5

In [31]:
# Equation (4) - single viewing scenario
energyMobileC = intensityMobile * durationStreaming * dataTransmissionRate  # kWh
energyMobileC

0.30000000000000004

### Transmission - home router

Carbon Trust value = 71 Wh/hour.

In [32]:
energyHomeRouterC = float  # Energy consumption of the home router (E_HR)
intensityHomeRouter = (
    0.025  # Energy intensity of home router transmission (I_HR) kWh/GB (pg43, Table 3)
)

In [33]:
# Equation (6) - single viewing scenario
energyHomeRouterC = (
    intensityHomeRouter * durationStreaming * dataTransmissionRate
)  # kWh
energyHomeRouterCWh = energyHomeRouterC * 1000  # Wh
energyHomeRouterCWh  # Pg 52, Table 4 = 71 Wh/hour

75.00000000000001

### End-user devices

Carbon Trust value = 10 Wh/hour (TV Peripheral) + 86 Wh/hour (screens).

In [34]:
energyDevice = float  # Energy consumption of the device (E_VD)
# powerScreens = 22 # Average power consumption of screens (W_s) W (pg 101, Table 10)
powerScreens = 86  # W (pg 52, Table 4)
# powerPeripherals = 0 # Average power consumption of peripherals (W_p) W (pg 101, Table 10)
powerPeripherals = 10  # W (pg 52, Table 4)
# proportionScreens = ? # Proportion of viewing time at W_s,i relative to the entire streaming service
# proportionPeripherals = ? # Proportion of viewing time at W_p,i relative to the entire streaming service

In [35]:
energyDevice = (powerScreens + powerPeripherals) * durationStreaming
energyDevice  # Wh/hour = 96 (pg 52, Table 4)

96

### Totals

Carbon Trust value = 188 Wh/hour.

In [36]:
# energyVideoStreamingC = dataCenterEnergy + energyFixedC + energyMobileC + energyHomeRouterC + energyDevice
energyVideoStreamingC = (
    dataCenterEnergy + energyFixedCWh + energyHomeRouterCWh + energyDevice
)
energyVideoStreamingC  # Wh/hour = 188 (pg 52, Table 4)

191.8

## Power model

Results not provided, but the report says: "using the power models for fixed
networks accounted for approximately 60% of the total measured energy expected"
(pg 66).

### Global parameters

In [37]:
idleTimeFactor = 3  # Idle time allocation factor (F)

qtyActiveDevices = float  # Quantity of active devices per fixed line connection (Q_A)
qtyUsers = 2.3  # Quantity of users per fixed line (Q_U) - UK (pg 101, Table 11)
qtyDevices = 6.9  # Quantity of devices per user (Q_D) - UK (pg 101, Table 11)

activeDeviceFactor = 0.5  # Active device factor as a proportion of total devices (A)

In [38]:
qtyActiveDevices = qtyUsers * qtyDevices * activeDeviceFactor
qtyActiveDevices

7.935

### Transmission - fixed network

Carbon Trust value = 12 Wh/hour (60% of the conventional value, assumption TBC).

In [39]:
energyFixedP = float  # Energy consumption over the fixed network (E_FN)
baseloadFixedCore = 1.5  # Baseload fixed core network element per fixed line connection (B_FN,C) W/line (pg 62, Table 5)
baseloadFixedAccess = 5  # Baseload fixed access network element per fixed line connection (B_FN,A) W/line (pg 62, Table 5)
dynamicFixedCore = (
    0.03  # Dynamic fixed core network component (V_FN,C) W/Mbps (pg 62, Table 5)
)
dynamicFixedAccess = (
    0.02  # Dynamic fixed access network component (V_FN,A) W/bitrate% (pg 62, Table 5)
)
bandwidthFixed = 75  # Bandwidth of the fixed line connection (S) Mbps (pg 62, Table 5)

In [40]:
energyFixedP = (
    ((baseloadFixedCore + baseloadFixedAccess) / qtyActiveDevices) * idleTimeFactor
    + (dynamicFixedCore + (dynamicFixedAccess / bandwidthFixed) * 100)
    * dataTransmissionRate
) * durationStreaming
energyFixedP  # Wh

2.6274669187145556

### Transmission - mobile network

In [41]:
energyMobileP = float  # Energy consumption over the mobile network (E_MN)
baseloadMobileCore = 0.2  # Baseload mobile core network element per subscriber (B_MN,C) W/subscriber (pg 62, Table 5)
baseloadMobileAccess = 1  # Baseload mobile access network element per subscriber (B_MN,A) W/subscriber (pg 62, Table 5)
dynamicMobileCore = (
    0.03  # Dynamic mobile core network dynamic element (V_MN,C) W/Mbps (pg 62, Table 5)
)
dynamicMobileAccess = 1.5  # Dynamic mobile access network dynamic element (V_MN,A) W/Mbps (pg 62, Table 5)

In [42]:
energyMobileP = (
    baseloadMobileCore
    + baseloadMobileAccess
    + (dynamicMobileCore + dynamicMobileAccess) * dataTransmissionRate
) * durationStreaming

energyMobileP

5.79

### Transmission - home router

Carbon Trust value = 42.6 Wh/hour (60% of the conventional value, assumption
TBC).

In [43]:
energyHomeRouterP = float  # Energy consumption of the home router (E_HR)
baseloadHomeRouter = (
    10  # Baseload power consumption of the home router (B_HR) W/line (pg 62, Table 5)
)

In [44]:
energyHomeRouterP = (
    (baseloadHomeRouter / qtyActiveDevices) * idleTimeFactor * durationStreaming
)
energyHomeRouterP  # Wh

3.7807183364839316

### Totals

Carbon Trust value = 151.6 Wh/hour (60% of the conventional value, assumption
TBC).

In [45]:
energyVideoStreamingP = (
    dataCenterEnergy + energyFixedP + energyHomeRouterP + energyDevice
)
energyVideoStreamingP  # Wh/hour

103.70818525519849

## Comparison

In [46]:
print(f"Con: Transmission (fixed) = {energyFixedCWh:,.1f} Wh/hour")
print(f"Pow: Transmission (fixed) = {energyFixedP:,.1f} Wh/hour")
print(f"Con: Home router = {energyHomeRouterCWh:,.1f} Wh/hour")
print(f"Pow: Home router = {energyHomeRouterP:,.1f} Wh/hour")
print(f"Con: Total = {energyVideoStreamingC:,.1f} Wh/hour")
print(f"Pow: Total = {energyVideoStreamingP:,.1f} Wh/hour")

Con: Transmission (fixed) = 19.5 Wh/hour
Pow: Transmission (fixed) = 2.6 Wh/hour
Con: Home router = 75.0 Wh/hour
Pow: Home router = 3.8 Wh/hour
Con: Total = 191.8 Wh/hour
Pow: Total = 103.7 Wh/hour
